In [1]:
import os
from google.colab import output
from google.colab import files

file_selector = files.upload()

if not file_selector:
  print("No file selected.")

uploaded_file = next(iter(file_selector.values()))

try:
  if not uploaded_file.name.endswith(".xlsx"):
    print("Please select an .xlsx file.")
except:
  print("Please check file type")

with open("/content/data.xlsx", "wb") as f:
  f.write(uploaded_file)

# Print a success message
print("File uploaded and saved as data.xlsx.")



Saving data.xlsx to data.xlsx
Please check file type
File uploaded and saved as data.xlsx.


In [2]:
!mkdir "/content/Model/"
!clear
!wget https://github.com/ujwalnk/CIE-Candidate-Selection/raw/main/Models/model-bert-use-this.hdf5
!clear
!mv "/content/model-bert-use-this.hdf5" "/content/Model/model-bert-use-this.hdf5"
!clear
!wget https://github.com/ujwalnk/CIE-Candidate-Selection/raw/main/Models/model_weights_and_optimizer.pickle
!clear
!mv "/content/model_weights_and_optimizer.pickle" "/content/Model/model_weights_and_optimizer.pickle"
!clear
!wget https://github.com/ujwalnk/CIE-Candidate-Selection/raw/main/Models/random_forest_model.joblib
!clear
!mv "/content/random_forest_model.joblib" "/content/Model/random_forest_model.joblib"
!clear
!wget https://github.com/ujwalnk/CIE-Candidate-Selection/raw/main/Models/vectorizer.joblib
!clear
!mv "/content/vectorizer.joblib" "/content/Model/vectorizer.joblib"
!clear

--2024-04-19 16:30:01--  https://github.com/ujwalnk/CIE-Candidate-Selection/raw/main/Models/model-bert-use-this.hdf5
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ujwalnk/CIE-Candidate-Selection/main/Models/model-bert-use-this.hdf5 [following]
--2024-04-19 16:30:01--  https://raw.githubusercontent.com/ujwalnk/CIE-Candidate-Selection/main/Models/model-bert-use-this.hdf5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 418600 (409K) [application/octet-stream]
Saving to: ‘model-bert-use-this.hdf5’

model-bert-use-this 100%[===================>] 408.79K  --.-KB/s    in 0.04s   

2024-04-19 16:30:01

Verify that the excel file has been added to the current directory.

> Need to verify later if the pickle files of both @Sairam & @Swastik are added to the current directory

In [3]:
!echo "Please Be Patient"
!pip install tensorflow_text -q
!pip install --upgrade tf-keras -q
!pip install aspose-cells -q
!echo "Thank You For Your Patience"

Please Be Patient
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 24.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 33.3 M

In [4]:
import pandas as pd


full_dataframe = pd.read_excel('/content/data.xlsx')

In [5]:
# Sairam 7 questions

questions = [
    "Give one/top reason why you want to take up this course",
    "What (one/top thing) do you expect to get out of this course?",
    "What does entrepreneurship mean to you?",
    "Can you give one example of any prototype you have built?",
    "Optional question: Are you excited to work on any ideas/opportunities as part of this course. If so please elaborate."
  ]

AI Component Detection Below

In [6]:
#run this cell once before predicting values (loading the model)
#Accuracy of the model = 82-86%
import tensorflow
import pickle
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text as text  # Registers the ops.

import numpy as np


class BertEncoder(tensorflow.keras.layers.Layer):
    def __init__(self, preprocess_path, model_path, **kwargs):
        super(BertEncoder, self).__init__(**kwargs)
        self.preprocessor = hub.KerasLayer(preprocess_path)
        self.encoder = hub.KerasLayer(model_path, trainable=True)

    def call(self, inputs):
        encoder_inputs = self.preprocessor(inputs)
        outputs = self.encoder(encoder_inputs)
        pooled_output = outputs['pooled_output']  # [batch_size, 512].
        sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 512].
        return pooled_output, sequence_output

with tensorflow.keras.utils.custom_object_scope({'BertEncoder': BertEncoder}):
    # Load the model architecture and weights
    model = tensorflow.keras.models.load_model('/content/Model/model-bert-use-this.hdf5')

# Load model weights and optimizer state from pickle file
with open('/content/Model/model_weights_and_optimizer.pickle', 'rb') as f:
    weights, optimizer_config = pickle.load(f)

# Set model weights
model.set_weights(weights)

# Reconstruct optimizer from config
optimizer = tensorflow.keras.optimizers.Adam.from_config(optimizer_config)

print("Model Loaded")

Model Loaded


In [7]:
y_main=[]
for i in questions:
  temp1=[]
  y_pred=model.predict(full_dataframe[i].fillna("NA"))
  for j in y_pred:
    temp1.append(j[0])
  y_pred
  y_main.append(temp1)

7/7 ━━━━━━━━━━━━━━━━━━━━ 26s 4s/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step


In [8]:
y_main = [list(i) for i in zip(*y_main)]#transposing the matrix
ai_df=pd.DataFrame(y_main)

Natural Language Processing Below

In [9]:
# Your output df will be like this I presume, please use the same variable names.
import numpy as np

import pandas as pd
from joblib import load

# Load the vectorizer
loaded_vectorizer = load('/content/Model/vectorizer.joblib')

# Load the model
loaded_model = load('/content/Model/random_forest_model.joblib')

# Read the data from data.xlsx
data_df = full_dataframe #pd.read_excel('/content/CIE L1_Aug2023 semester_OAT data.xlsx')

# Assuming the text responses are in columns 7 onwards
text_columns = data_df.columns[7:]

# Iterate over each row and make predictions
for index, row in data_df.iterrows():
    # Concatenate all text responses into a single string https://www.canva.com/design/DAGCG5bJlhs/ZaWlO8EVoVCsPdDpJqnM0A/edit?utm_content=DAGCG5bJlhs&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton
    answers_row = ' '.join(row[text_columns].astype(str))

    # Transform the text using the loaded vectorizer
    answers_row_vec = loaded_vectorizer.transform([answers_row])

    # Make prediction for the row
    prediction = loaded_model.predict(answers_row_vec)

    # Print the predicted status
    predicted_status = 'confirmed' if prediction[0] == 1 else 'not confirmed'

           0
0   0.352919
1   0.135857
2   0.992115
3   0.739386
4   0.340593
..       ...
95  0.970884
96  0.372537
97  0.049686
98  0.059132
99  0.862416

[100 rows x 1 columns]


In [10]:
import pandas as pd
from joblib import load

# Load the vectorizer and model
loaded_vectorizer = load('/content/Model/vectorizer.joblib')
loaded_model = load('/content/Model/random_forest_model.joblib')

# Assume data_df is already defined and includes the required data
text_columns = data_df.columns[7:]

# List to store each row's data
data = []

# Iterate over rows in the dataframe
for index, row in full_dataframe.iterrows():
    # Vectorize the given row
    answers_row = ' '.join(row[text_columns].astype(str))
    answers_row_vec = loaded_vectorizer.transform([answers_row])

    # Predict the probability of the confirmed class
    probability = loaded_model.predict_proba(answers_row_vec)[:, 1]

    # Append dictionary to list
    data.append({'probability': float(probability[0])})

# Create DataFrame from list of dictionaries
results_df = pd.DataFrame(data)

# Optionally, to see the first few rows of your DataFrame:
# print(results_df.head())

nlp_df = results_df

   probability
0     0.304626
1     0.398175
2     0.348375
3     0.349435
4     0.295705


Outputting to the Excel File

In [11]:
import pandas as pd
import numpy as np


# I am dividing it here to normalize the score

ai_df.loc[:, "AI Component"] = ai_df.sum(axis=1) / 5
ai_df = ai_df[["AI Component"]]

temp_df = full_dataframe
temp_df = pd.concat([temp_df, ai_df, nlp_df], axis=1)
temp_df.to_excel("/content/output.xlsx", index=True)

print("Output File Created!")

Output File Created!


# Database

In [12]:
import sqlite3 as s
import datetime

def write_2_db(c, connection, param, value):
  c.execute("""
    INSERT INTO GRAPH_DATA (PARAM, YEAR, MONTH, VALUE) VALUES (?, ?, ?, ?)
  """, (param, datetime.date.today().year, datetime.date.today().month, value))

  # Save the transaction
  connection.commit()

connection = s.connect("/content/base.sqlite")

c = connection.cursor()

# Create a new table
c.execute("""
    CREATE TABLE IF NOT EXISTS GRAPH_DATA (
        PARAM TEXT NOT NULL,
        YEAR INT NOT NULL,
        MONTH INT NOT NULL,
        VALUE INT NOT NULL
    )
""")

In [13]:
write_2_db(c, connection, "AI", int(ai_df.mean()["AI Component"] * 100))

write_2_db(c, connection, "Branch/CS", int(full_dataframe["Branch"].value_counts()["CS"]))
write_2_db(c, connection, "Branch/EC", int(full_dataframe["Branch"].value_counts()["ECE"]))
write_2_db(c, connection, "Branch/EE", int(full_dataframe["Branch"].value_counts()["EEE"]))
write_2_db(c, connection, "Branch/AI", int(full_dataframe["Branch"].value_counts()["AIML"]))
write_2_db(c, connection, "Branch/BT", int(full_dataframe["Branch"].value_counts()["Biotech"]))
write_2_db(c, connection, "Branch/ME", int(full_dataframe["Branch"].value_counts()["Mech"]))

def get_year(x, y):
  sum = 0
  try:
    sum += int(full_dataframe["Semester"].value_counts()[x])
  except:
    try:
      sum += int(full_dataframe["Semester"].value_counts()[y])
    except:
      pass

  return sum

write_2_db(c, connection, "YEAR/1", int(get_year("1st","2nd")))
write_2_db(c, connection, "YEAR/2", int(get_year("3rd","4th")))
write_2_db(c, connection, "YEAR/3", int(get_year("5th","6th")))
write_2_db(c, connection, "YEAR/4", int(get_year("7th","8th")))

c.execute("SELECT * from GRAPH_DATA")
db_output = c.fetchall()

Putting the data into Excel file charts

In [14]:
import pandas as pd


# Convert data to DataFrame
df = pd.DataFrame(db_output, columns=['Parameter', 'Year', 'Month', 'Value'])

# Pivot the DataFrame to get the desired format
pivot_df = df.pivot_table(index='Parameter', columns=['Year', 'Month'], values='Value', aggfunc='sum', fill_value=0)

# Flatten the multi-level columns
pivot_df.columns = [f'{col[0]}-{col[1]}' for col in pivot_df.columns]

# Write the formatted data to Excel
excel_file_path = '/content/formatted_data.xlsx'
with pd.ExcelWriter(excel_file_path) as writer:
    pivot_df.to_excel(writer, sheet_name='Formatted Data')

print(f'Formatted data has been written to {excel_file_path}')


Formatted data has been written to /content/formatted_data.xlsx


In [15]:
#AI COMPONENT GRAPH
import jpype
import asposecells
jpype.startJVM()
from asposecells.api import Workbook, ChartType, FileFormatType

workbook = Workbook("/content/formatted_data.xlsx")
worksheet = workbook.getWorksheets().get(0)

data_range=len([str(db_output[x][1])  + "-" + str(db_output[x][2]) for x in range(0, len(db_output), 11)])

chartIndex = worksheet.getCharts().add(ChartType.COLUMN, 6, 2, 22, 10)
chart = worksheet.getCharts().get(chartIndex)
chart.setChartDataRange(f"A1:{chr(ord('A')+data_range)}2", True)

chartIndex_branch = worksheet.getCharts().add(ChartType.COLUMN, 6, 20, 22, 30)
chart_branch = worksheet.getCharts().get(chartIndex_branch)
chart_branch.setChartDataRange(f"A3:C8", True)

chartIndex_year = worksheet.getCharts().add(ChartType.COLUMN, 6, 15, 22, 25)
chart_year = worksheet.getCharts().get(chartIndex_year)
chart_year.setChartDataRange(f"A9:C12", True)

# Save the Excel file
workbook.save("excel-column-chart.xlsx")

In [16]:
from google.colab import files

!echo "Please allow multiple files downloads if all files are not getting downloaded"

files.download('/content/excel-column-chart.xlsx')
files.download('/content/output.xlsx')
files.download('/content/base.sqlite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>